# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
print(data.shape)

(21525, 12)


In [3]:
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

In [4]:
print(data.tail(10))

       children  days_employed  dob_years       education  education_id  \
21515         1    -467.685130         28         среднее             1   
21516         0    -914.391429         42          высшее             0   
21517         0    -404.679034         42          высшее             0   
21518         0  373995.710838         59         СРЕДНЕЕ             1   
21519         1   -2351.431934         37  ученая степень             4   
21520         1   -4529.316663         43         среднее             1   
21521         0  343937.404131         67         среднее             1   
21522         1   -2113.346888         38         среднее             1   
21523         3   -3112.481705         38         среднее             1   
21524         2   -1984.507589         40         среднее             1   

          family_status  family_status_id gender income_type  debt  \
21515   женат / замужем                 0      F   сотрудник     1   
21516   женат / замужем           

**Вывод**

После изучения данных видим, что во многих строках существуют нулевые значения. Также, заметно, что столбец days_employed содержит большое количество отрицательных значений. Вероятно когда речь идет о днях, то точность до 6 знака после запятой будет не обязательна. И видим разность строчных и заглавных букв в образовании.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [6]:
print(data['days_employed'].isnull().sum())

2174


In [7]:
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [8]:
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [9]:
print(data['education_id'].value_counts())

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


In [10]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [11]:
print(data['family_status_id'].value_counts())

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


In [12]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [13]:
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64


In [14]:
print(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


In [15]:
print(data['total_income'].isna().sum())

2174


In [16]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [17]:
print(data[data['gender']=='XNA'])

       children  days_employed  dob_years            education  education_id  \
10701         0   -2358.600502         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10701  гражданский брак                 1    XNA   компаньон     0   

        total_income               purpose  
10701  203905.157261  покупка недвижимости  


Пропущенное значение в столбце gender, не влияет на данные, все остальные значения не являются единичными в выборке.Поэтому можно его удалить.

In [18]:
data = data.drop(data[data['gender']=='XNA'].index)
print(data['gender'].value_counts())

F    14236
M     7288
Name: gender, dtype: int64


In [19]:
print(data[data['dob_years']==0])

       children  days_employed  dob_years education  education_id  \
99            0  346541.618895          0   Среднее             1   
149           0   -2664.273168          0   среднее             1   
270           3   -1872.663186          0   среднее             1   
578           0  397856.565013          0   среднее             1   
1040          0   -1158.029561          0    высшее             0   
...         ...            ...        ...       ...           ...   
19829         0            NaN          0   среднее             1   
20462         0  338734.868540          0   среднее             1   
20577         0  331741.271455          0   среднее             1   
21179         2    -108.967042          0    высшее             0   
21313         0   -1268.487728          0   среднее             1   

               family_status  family_status_id gender income_type  debt  \
99           женат / замужем                 0      F   пенсионер     0   
149                в 

 Рассмотрим пропуски в столбце dob_years, для начала рассмотрим пропуски по типу заннятости.

In [20]:
print(data.groupby('income_type')['dob_years'].median())

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64


In [21]:
data['dob_years'] = data.groupby('income_type')['dob_years'].transform(lambda x : x.replace(0,x.median()))
print(data[data['dob_years']==0])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


Заменив нулевые значения в столбце возраста, на медийные значения по типу занятости, избавились от пропущенных значений,
не изменяя остальные столбцы.

Теперь можно убрать нулевые значения в столбце с трудовым стажем для начала , необходимо избавится от минусов в данном столбце.

In [22]:
def delit_minus(row):
    if row['days_employed']<0:
        return row['days_employed']*(-1)
    return row['days_employed']
data['days_employed'] = data.apply(delit_minus,axis = 1)
print(data.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

In [23]:
print(data.groupby('dob_years')['days_employed'].mean())

dob_years
19       633.678086
20       684.944308
21       709.440930
22      2793.321854
23       827.309437
24      1020.900547
25      1088.406453
26      3214.617245
27      3725.820728
28      2180.056457
29      1553.823200
30      1696.039355
31      2331.246721
32      3877.197466
33      3241.413096
34      4141.242883
35      2755.226484
36      4496.534938
37      5879.263440
38      7641.485465
39      4885.402566
40      7063.431634
41      6400.926359
42      8885.465110
43      9418.428468
44      9952.060042
45     11748.039481
46     14121.191502
47     14223.685900
48     18149.683912
49     26679.790846
50     50493.031018
51     68425.772762
52     82803.402429
53     94073.249426
54    126757.781831
55    152264.946612
56    157417.505013
57    193689.068297
58    188571.489818
59    227496.974028
60    267695.441073
61    246731.048849
62    274381.283863
63    293383.721011
64    287670.475737
65    287448.538552
66    313613.718234
67    317629.788586
68    3268

Попробуем избавиться от пропусков заменяя медианой в возростной группе, будем разбивать примерно так, 
до 25 лет(примерно начало карьеры и студенты), от 25 до 35, от 35 до 45, от 45 до 55 и старше 55.

In [24]:
data['days_employed'] = data['days_employed'].fillna(0)

In [25]:
print(data[(data['dob_years'] >= 25) & (data['dob_years'] < 35)]['days_employed'].median())

1137.817856239775


In [26]:
def replace_days_employed(row):
    if row['days_employed'] == 0 :
        if row['dob_years'] <25:
            return data[data['dob_years'] < 55]['days_employed'].median()
        if row['dob_years'] >= 25 and row['dob_years'] < 35 :
            return data[(data['dob_years'] >= 25) & (data['dob_years'] < 35)]['days_employed'].median()
        if row['dob_years'] >= 35 and row['dob_years'] < 45 :
            return data[(data['dob_years'] >= 35) & (data['dob_years'] < 45)]['days_employed'].median()
        if row['dob_years'] >= 45 and row['dob_years'] < 55 :
            return data[(data['dob_years'] >= 45) & (data['dob_years'] < 55)]['days_employed'].median()
        if row['dob_years'] >= 55:
            return data[data['dob_years'] >= 55]['days_employed'].median()
    return row['days_employed']
data['days_employed'] = data.apply(replace_days_employed,axis = 1)
print(data[data['days_employed']==0])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [27]:
print(data.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

Заменили пропуски в столбце с трудовым стажем, следующий столбец с пропусками информация о ежемесячном доходе

In [28]:
data_dob_25 = data[data['dob_years']< 25]
data_dob_35 = data[(data['dob_years'] >= 25) & (data['dob_years'] < 35)]
print(data_dob_25['total_income'].median())
print('-------------------------------------------')
print(data_dob_25.groupby('income_type')['total_income'].median())
print('----------------------------------------------')
print(data_dob_35['total_income'].median())
print('-------------------------------------------')
print(data_dob_35.groupby('income_type')['total_income'].median())

128479.29359755457
-------------------------------------------
income_type
госслужащий    133510.813687
компаньон      138681.737450
пенсионер       89368.600062
сотрудник      122938.149557
студент         98201.625314
Name: total_income, dtype: float64
----------------------------------------------
149943.13240890088
-------------------------------------------
income_type
безработный         59956.991984
госслужащий        152726.500906
компаньон          170123.325731
пенсионер           96116.021663
предприниматель    499163.144947
сотрудник          142014.190010
Name: total_income, dtype: float64


В столбце дохода есть большой разрыв ,между минимальным и максимальным доходом, поэтому заполнять будем медианами, также есть разница в доходах по типу занятости. Будем заменять опираясь на возраст и тип занятости.

In [30]:

dob_last = 0
for dob in [25,35,45,55,100]:
    data_dob = data[(data['dob_years'] >= dob_last) & (data['dob_years'] < dob)]
    data_dob_group = data_dob.groupby('income_type').transform(lambda x : x.fillna(x.mean()))
    data.loc[(data['dob_years'] >= dob_last) & (data['dob_years'] < dob),'total_income'] = data_dob_group
    dob_last = dob
print(data['total_income'].isna().sum())

1


In [31]:

data.dropna(subset=['total_income'],inplace=True)
print(data['total_income'].isna().sum())

0


In [32]:
print(data[data['total_income']== 0])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [34]:
data = data.drop(data[data['total_income']==0].index)
print(data[data['total_income']== 0])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [36]:
print(data.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

**Вывод**

Заменили пропуски найденные во время ознакомления с данными, данный датасет не готов окончательно к анализу, есть недостатки в виде дубликатов, разных регистров, и перефразированных целей кредита.

### Замена типа данных

Столбцы с трудовым стажем и ежемесячным доходом, содержат тип с плавающей запятой, такая точность для анализа не пригодится, сделаем их целочисленными.

In [37]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
print(data.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1           8437         42               высшее             0   
1          1           4024         36              среднее             1   
2          0           5623         33              Среднее             1   
3          3           4124         32              среднее             1   
4          0         340266         53              среднее             1   
5          0            926         27               высшее             0   
6          0           2879         43               высшее             0   
7          0            152         50              СРЕДНЕЕ             1   
8          2           6929         35               ВЫСШЕЕ             0   
9          0           2188         41              среднее             1   
10         2           4171         36               высшее             0   
11         0            792         40              среднее             1   

В столбце с количеством детей, есть ошибка, есть отрицательное количество детей ,что скорее всего стало ошибкой при получении данных.

In [38]:
data['children'] = data['children'].replace(-1,1)
print(data['children'].value_counts())

0     14147
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


**Вывод**

Заменили все числа с плавающей запятой на целочисленные.

### Обработка дубликатов

In [39]:
print(data.duplicated().sum())

54


Метод duplicated(), нашел 54 дубликата, допустимо их удалить методом drop_duplicated() с восстановлением индексации. Дубликаты могли появится, при полном соответсвии данных, у заемщиков, или при неоднократном их обращении.

In [40]:
data = data.drop_duplicates().reset_index(drop=True) 
print(data.duplicated().sum())

0


**Вывод**

Нашли полные совпадения в данных и удалили их.

### Лемматизация

Для начала приведем к одному регистру информацию об образовании.

In [41]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15188
высшее                  5250
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64


Проверим ещё раз на дубликаты, и удалим найденные.

In [42]:
print(data.duplicated().sum())

17


In [43]:
data = data.drop_duplicates().reset_index(drop=True) 
print(data.duplicated().sum())

0


Подготовим данные к группировке, при помощи лематизации проведем цели кредита к более конкретным.

In [44]:

m = Mystem() 

In [45]:
for row in data['purpose'].unique():
    lema = m.lemmatize(row)
    print(lema)
    

['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['жилье', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['автомобиль', '\n']
['заниматься', ' ', 'образование'

In [46]:
def lema_purpose(row):
    lema = m.lemmatize(row['purpose'])
    if 'свадьба' in lema:
        return 'свадьба'
    if 'автомобиль' in lema:
        return 'автомобиль'
    if 'образование'in lema:
        return 'образование'
    if 'жилье' in lema or 'недвижимость'in lema:
        return 'недвижимость'
    return 'другое'
data['lem_purpose'] = data.apply(lema_purpose,axis=1)
print(data['lem_purpose'].value_counts())

недвижимость    10809
автомобиль       4306
образование      4013
свадьба          2324
Name: lem_purpose, dtype: int64


**Вывод**

Провели лемматизацию данных по целям кредита и привели к нижнему регистру по образованиям, и удалили ещё 17 дубликатов.

### Категоризация данных

Скатегорируйем данные таким образом, чтобы было легче отвечать на поставленные для анализа вопросы. Для начала добавим столбец который поможет проанализорвать количество детей,пусть столбец характеризует наличее детей, есть или нет.

In [47]:
def apply_children(row):
    if row['children'] > 0 :
        return 'есть'
    return 'нет'
data['having_children'] = data.apply(apply_children,axis = 1)
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                      purpose   lem_purpose having_children  
0               покупка жилья  недвижимость            есть  
1   

Введем категорию по доходу, будем ориентироваться на средний доход, дадим выборки ниже среднего и выше среднего для этого вернемся к фильтру по возрасту и типу занятости.

In [48]:
dob_last = 0
data_income_mean = []
data_income_mean_columns = ['dob','income_type','mean']
for dob in [25,35,45,55,100]:
    data_dob = data[(data['dob_years'] >= dob_last) & (data['dob_years'] < dob)]
    for types in data_dob['income_type'].unique():
        data_income_mean.append([dob,types,data_dob[data_dob['income_type'] == types]['total_income'].mean()])
    dob_last = dob
df = pd.DataFrame(data = data_income_mean, columns = data_income_mean_columns )
def category_total_income(row):
    types = row['income_type']
    if row['dob_years'] < 25:
        for string in data_income_mean:
            if string[0]== 25 and string[1]== types and string[2]>row['total_income']:
                return 'ниже среднего'
            if string[0]== 25 and string[1]== types and string[2]<row['total_income']:
                return 'выше среднего'
    elif row['dob_years'] < 35:
        for string in data_income_mean:
            if string[0]==35 and string[1]==types and string[2]>row['total_income']:
                return 'ниже среднего'
            if string[0]==35 and string[1]==types and string[2]<row['total_income']:
                return 'выше среднего'
    elif row['dob_years'] < 45:
        for string in data_income_mean:
            if string[0]==45 and string[1]==types and string[2]>row['total_income']:
                return 'ниже среднего'
            if string[0]==45 and string[1]==types and string[2]<row['total_income']:
                return 'выше среднего'
    elif row['dob_years'] < 55:
        for string in data_income_mean:
            if string[0]==55 and string[1]==types and string[2]>row['total_income']:
                return 'ниже среднего'
            if string[0]==55 and string[1]==types and string[2]<row['total_income']:
                return 'выше среднего'
    elif row['dob_years'] > 55:
        for string in data_income_mean:
            if string[0]==100 and string[1]==types and string[2]>row['total_income']:
                return 'ниже среднего'
            if string[0]==100 and string[1]==types and string[2]<row['total_income']:
                return 'выше среднего'
    #return 'error'
            
data['category_income'] = data.apply(category_total_income,axis = 1)
print(data.tail(15))

       children  days_employed  dob_years            education  education_id  \
21437         2           1137         28              среднее             1   
21438         0            612         29               высшее             0   
21439         0            165         26               высшее             0   
21440         0           1166         35              среднее             1   
21441         0            280         27  неоконченное высшее             2   
21442         1            467         28              среднее             1   
21443         0            914         42               высшее             0   
21444         0            404         42               высшее             0   
21445         0         373995         59              среднее             1   
21446         1           2351         37       ученая степень             4   
21447         1           4529         43              среднее             1   
21448         0         343937         6

In [49]:
print(data['category_income'].value_counts())

ниже среднего    12834
выше среднего     8168
Name: category_income, dtype: int64


**Вывод**

Мы добавили категориальные признаки, при помощи которых можно ответить на поставленные вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот и следующие вопросы вычислим конверсии.

In [50]:
children_debt = data.groupby('having_children')['debt'].value_counts()

print(children_debt)

conv_children_debt = children_debt[1] / len(data['debt'])
conv_children_nodebt = children_debt[0] / len(data['debt'])
conv_nochildren_debt = children_debt[3] / len(data['debt'])
conv_nochildren_nodebt = children_debt[2] / len(data['debt'])
print('Клиентов с детьми вернувшись кредит в срок в {:.0f} раз больше'.format(conv_children_nodebt / conv_children_debt))
print('Клиентов без детей вернувшись кредит в срок в {:.0f} раз больше'.format(conv_nochildren_nodebt / conv_nochildren_debt))

having_children  debt
есть             0        6685
                 1         678
нет              0       13026
                 1        1063
Name: debt, dtype: int64
Клиентов с детьми вернувшись кредит в срок в 10 раз больше
Клиентов без детей вернувшись кредит в срок в 12 раз больше


**Вывод**

Можно сделать вывод, что клиенты без детей возвращают кредит в срок чаще.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [51]:
family_debt = data.groupby('family_status')['debt'].value_counts()
print(family_debt)
conv_holost_nodebt = family_debt[0] / len(data['family_status'])
conv_holost_debt = family_debt[1] / len(data['family_status'])
conv_razvod_nodebt = family_debt[2] / len(data['family_status'])
conv_razvod_debt = family_debt[3] / len(data['family_status'])
conv_widow_nodebt = family_debt[4] / len(data['family_status'])
conv_widow_debt = family_debt[5] / len(data['family_status'])
conv_brak_nodebt = family_debt[6] / len(data['family_status'])
conv_brak_debt = family_debt[7] / len(data['family_status'])
conv_marryed_nodebt = family_debt[8] / len(data['family_status'])
conv_marryed_debt = family_debt[9] / len(data['family_status'])

print('Доля не женатых клиентов вернувших кредит в срок в {:.0f} раз больше'.format(conv_holost_nodebt / conv_holost_debt))
print('Доля разведенных клиентов вернувших кредит в срок в {:.0f} раз больше'.format(conv_razvod_nodebt / conv_razvod_debt))
print('Доля клиентов в гражданском браке вернувших кредит в срок в {:.0f} раз больше'.format(conv_brak_nodebt / conv_brak_debt))
print('Доля женатых клиентов вернувших кредит в срок в {:.0f} раз больше'.format(conv_marryed_nodebt / conv_marryed_debt))
print('Доля клиентов вдова/вдовец вернувших кредит в срок в {:.0f} раз больше'.format(conv_widow_nodebt / conv_widow_debt))

family_status          debt
Не женат / не замужем  0        2536
                       1         274
в разводе              0        1110
                       1          85
вдовец / вдова         0         896
                       1          63
гражданский брак       0        3762
                       1         388
женат / замужем        0       11407
                       1         931
Name: debt, dtype: int64
Доля не женатых клиентов вернувших кредит в срок в 9 раз больше
Доля разведенных клиентов вернувших кредит в срок в 13 раз больше
Доля клиентов в гражданском браке вернувших кредит в срок в 10 раз больше
Доля женатых клиентов вернувших кредит в срок в 12 раз больше
Доля клиентов вдова/вдовец вернувших кредит в срок в 14 раз больше


**Вывод**

Клиенты с семейным положением вдова/вдовец, чаще возвращают кредит в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [52]:
category_debt = data.groupby('category_income')['debt'].value_counts()
print(category_debt)
conv_up_nodebt = category_debt[0] / len(data['category_income'])
conv_up_debt = category_debt [1] / len(data['category_income'])
conv_down_nodebt = category_debt[2] / len(data['category_income'])
conv_down_debt = category_debt [3] / len(data['category_income'])
print('Доля клиентов с доходом ниже среднего, вернувших кредит в срок в {:.0f} раз больше'.format(conv_down_nodebt / conv_down_debt))
print('Доля клиентов с доходом выше среднего, вернувших кредит в срок в {:.0f} раз больше'.format(conv_up_nodebt / conv_up_debt))
#print('Конверсия для клиентов без долгов с доходом ниже среднего: {:.2%}'.format(conv_nizhe))
#print('Конверсия для клиентов без долгов с доходом выше среднего: {:.2%}'.format(conv_vyshe))


category_income  debt
выше среднего    0        7554
                 1         614
ниже среднего    0       11732
                 1        1102
Name: debt, dtype: int64
Доля клиентов с доходом ниже среднего, вернувших кредит в срок в 11 раз больше
Доля клиентов с доходом выше среднего, вернувших кредит в срок в 12 раз больше


**Вывод**

Клиенты с доходом выше среднего имеют меньше долгов.

- Как разные цели кредита влияют на его возврат в срок?

In [53]:
purpose_debt = data.groupby('lem_purpose')['debt'].value_counts()
print(purpose_debt)
conv_avto_nodebt = purpose_debt[0] / len(data['lem_purpose'])
conv_avto_debt = purpose_debt[1] / len(data['lem_purpose'])
conv_nedv_nodebt = purpose_debt[2] / len(data['lem_purpose'])
conv_nedv_debt = purpose_debt[3] / len(data['lem_purpose'])
conv_obraz_nodebt = purpose_debt[4] / len(data['lem_purpose'])
conv_obraz_debt = purpose_debt[5] / len(data['lem_purpose'])
conv_mar_nodebt = purpose_debt[6] / len(data['lem_purpose'])
conv_mar_debt = purpose_debt[7] / len(data['lem_purpose'])
print('Доля клиентов, бравших кредит на авто, вернувших кредит в срок в {:.0f} раз больше'.format(conv_avto_nodebt / conv_avto_debt))
print('Доля клиентов, бравших кредит на недвижимость, вернувших кредит в срок в {:.0f} раз больше'.format(conv_nedv_nodebt / conv_nedv_debt))
print('Доля клиентов, бравших кредит на образование, вернувших кредит в срок в {:.0f} раз больше'.format(conv_obraz_nodebt / conv_obraz_debt))
print('Доля клиентов, бравших кредит на свадьбу, вернувших кредит в срок в {:.0f} раз больше'.format(conv_mar_nodebt / conv_mar_debt))


lem_purpose   debt
автомобиль    0        3903
              1         403
недвижимость  0       10027
              1         782
образование   0        3643
              1         370
свадьба       0        2138
              1         186
Name: debt, dtype: int64
Доля клиентов, бравших кредит на авто, вернувших кредит в срок в 10 раз больше
Доля клиентов, бравших кредит на недвижимость, вернувших кредит в срок в 13 раз больше
Доля клиентов, бравших кредит на образование, вернувших кредит в срок в 10 раз больше
Доля клиентов, бравших кредит на свадьбу, вернувших кредит в срок в 11 раз больше


**Вывод**

Клиенты бравшие кредит на недвижимость возвращали долги чаще остальных, а люди, которые брали кредит на образование и авто, возвращали одинаково.

## Шаг 4. Общий вывод

Провнализировали данные, нашли ошибки в предоставленных данных, заполнили пропуски в данных, заменили тип данных. Ответили на поставленные вопросы, зависимость прослеживается между возвратом кредита и доходом, а также целью кредита.